<a href="https://colab.research.google.com/github/deokwoo-han/abc/blob/master/10_06_project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from itertools import combinations, groupby
from collections import Counter
from IPython.display import display
color = sns.color_palette()

%matplotlib inline
%config InlineBackend.figure_config = 'retina'

pd.options.mode.chained_assignment = None  # default='warn'

###1. 구글 드라이버 파일 가져오기(시행자 경로에 해당 파일이 있어야 함)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! pwd cd

/content


###2. 파일 불러와서 데이터 프레임으로 만들기

In [4]:
file_path = './drive/MyDrive/datas'
order_products_train = pd.read_csv(file_path + "/order_products__train.csv")
order_products_prior = pd.read_csv(file_path + "/order_products__prior.csv")
orders = pd.read_csv(file_path + "/orders.csv")
products = pd.read_csv(file_path + "/products.csv")
aisles = pd.read_csv(file_path + "/aisles.csv")
departments = pd.read_csv(file_path + "/departments.csv")

###3. 데이터 프레임 통합

In [5]:
order_products_prior = pd.merge(order_products_prior, products, on='product_id', how='left')
order_products_prior = pd.merge(order_products_prior, aisles, on='aisle_id', how='left')
order_products_prior = pd.merge(order_products_prior, departments, on='department_id', how='left')
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,2,33120,1,1,Organic Egg Whites,86,16,eggs,dairy eggs
1,2,28985,2,1,Michigan Organic Kale,83,4,fresh vegetables,produce
2,2,9327,3,0,Garlic Powder,104,13,spices seasonings,pantry
3,2,45918,4,1,Coconut Butter,19,13,oils vinegars,pantry
4,2,30035,5,0,Natural Sweetener,17,13,baking ingredients,pantry


###4. order_id와 department(제품 부서, 제품군)만 데이터프레임화

In [6]:
df2 = pd.DataFrame(order_products_prior.groupby('order_id').department.agg(lambda x: x.tolist())).reset_index()
df2.head(10)

,order_id,department
0,2,"[dairy eggs, produce, pantry, pantry, pantry, ..."
1,3,"[dairy eggs, dairy eggs, produce, produce, dai..."
2,4,"[bakery, personal care, snacks, breakfast, bre..."
3,5,"[produce, pantry, deli, produce, dairy eggs, h..."
4,6,"[beverages, household, household]"
5,7,"[beverages, frozen]"
6,8,[bakery]
7,9,"[produce, produce, produce, dairy eggs, snacks..."
8,10,"[produce, produce, produce, produce, produce, ..."
9,11,"[frozen, deli, canned goods, snacks, pantry]"


In [7]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

###5. 각 주문별 제품군 구매 여부

In [8]:
te = TransactionEncoder() 
te_result = te.fit(df2.department).transform(df2.department)
df2_1 = pd.DataFrame( te_result, columns = te.columns_)
df2_1.head()

,alcohol,babies,bakery,beverages,breakfast,bulk,canned goods,dairy eggs,deli,dry goods pasta,frozen,household,international,meat seafood,missing,other,pantry,personal care,pets,produce,snacks
0,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False
1,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False
2,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True
3,False,False,False,True,False,False,False,True,True,True,False,True,True,True,False,False,True,True,False,True,True
4,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False


###6. 제품군 구매 사례별 지지도

In [9]:
itemset1 = apriori(df2_1, use_colnames=True, min_support=0.03) 

In [10]:
itemset1

,support,itemsets
0,0.055278,(babies)
1,0.274212,(bakery)
2,0.453315,(beverages)
3,0.163362,(breakfast)
4,0.211923,(canned goods)
...,...,...
718,0.031293,"(snacks, dry goods pasta, produce, beverages, ..."
719,0.045762,"(snacks, produce, beverages, frozen, dairy egg..."
720,0.032411,"(canned goods, snacks, produce, frozen, dairy ..."
721,0.034251,"(snacks, produce, frozen, dairy eggs, deli, pa..."


### 7. 각 제품군 간의 연관 규칙 분석(conviction 순)

antecedents: 비교할 선행 제품군들

consequents: 비교할 후행 제품군들

antecedent support: 전체 주문 대비 선행 제품군들 구매비율(지지도)

consequent support: 전체 주문 대비 후행 제품군들 구매비율(지지도)

support: 두 제품군 동시 구매 비율(지지도)

confidence: 선행 제품군이 있을 때 후행 제품군도 있는 비율(조건부 확률, 신뢰도)

lift: 동시 구매 여부를 알기 위해 서로 간의 연관성을 파악하는 향상도(1보다 높을 경우 itemA, itemB 두 품목 간의 긍정적 관계, 낮을 경우 두 품목 간의 부정적 관계, 1일 경우 관계 없음)

leverage: 0에 가깝다면 각각 독립일 가능성이 높음. 양수면 향상도가 1보다 큰 경우와 유사, 음수면 향상도가 1보다 작은 경우와 유사

conviction: 독립 확률 지표. 1보다 크면 상호연관성 높음. 1보다 작으면 상호 연관성 낮음. 1이면 상호연관성 없음(독립)



In [11]:
df2_2 = association_rules(itemset1, metric="confidence", min_threshold=0.85 )
df2_2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(canned goods),(produce),0.211923,0.749429,0.186306,0.879123,1.173057,0.027485,2.072944
1,(deli),(produce),0.239605,0.749429,0.206059,0.859995,1.147533,0.026492,1.789726
2,(dry goods pasta),(produce),0.185967,0.749429,0.161743,0.869737,1.160533,0.022373,1.923585
3,(international),(produce),0.068910,0.749429,0.059938,0.869805,1.160624,0.008295,1.924583
4,(meat seafood),(produce),0.178772,0.749429,0.156475,0.875276,1.167923,0.022498,2.008996
...,...,...,...,...,...,...,...,...,...
561,"(snacks, frozen, dairy eggs, deli, pantry)",(produce),0.036712,0.749429,0.034251,0.932971,1.244909,0.006738,3.738261
562,"(deli, snacks, pantry, frozen)","(dairy eggs, produce)",0.040275,0.550483,0.034251,0.850425,1.544869,0.012080,3.005287
563,"(snacks, dry goods pasta, produce, frozen, pan...",(dairy eggs),0.033124,0.677270,0.030741,0.928076,1.370319,0.008308,4.487121
564,"(snacks, dry goods pasta, frozen, dairy eggs, ...",(produce),0.033143,0.749429,0.030741,0.927545,1.237669,0.005903,3.458301


In [15]:
rules_final = df2_2.sort_values('lift', ascending=False)
rules_final.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
503,"(deli, dry goods pasta, pantry)","(dairy eggs, produce)",0.035375,0.550483,0.030457,0.860965,1.564017,0.010983,3.233125
506,"(deli, snacks, dry goods pasta)","(dairy eggs, produce)",0.041999,0.550483,0.035958,0.856172,1.555308,0.012839,3.125367
543,"(deli, snacks, bakery, frozen)","(dairy eggs, produce)",0.038470,0.550483,0.032851,0.853942,1.551258,0.011674,3.077660
500,"(deli, dry goods pasta, frozen)","(dairy eggs, produce)",0.039724,0.550483,0.033869,0.852617,1.548851,0.012002,3.049987
481,"(canned goods, deli, pantry)","(dairy eggs, produce)",0.039601,0.550483,0.033757,0.852432,1.548516,0.011957,3.046170
565,"(snacks, dry goods pasta, pantry, frozen)","(dairy eggs, produce)",0.036083,0.550483,0.030741,0.851967,1.547670,0.010878,3.036593
562,"(deli, snacks, pantry, frozen)","(dairy eggs, produce)",0.040275,0.550483,0.034251,0.850425,1.544869,0.012080,3.005287
367,"(frozen, bakery, breakfast, produce)",(dairy eggs),0.033578,0.677270,0.031396,0.935016,1.380566,0.008655,4.966314
369,"(snacks, bakery, breakfast, produce)",(dairy eggs),0.039241,0.677270,0.036592,0.932504,1.376856,0.010016,4.781444
541,"(snacks, produce, frozen, deli, bakery)",(dairy eggs),0.035234,0.677270,0.032851,0.932376,1.376668,0.008988,4.772400


### 8. 데이터프레임 CSV화

In [16]:
df2_2.to_csv('sample2.csv')

lift로 따져 봤을 때, dairy eggs 제품군에 속한 것을 다른 제품군과 같이 구매하는 경우가 많았다. 이를 볼 때, 해당 제품군을 기본으로 하여 타 제품군 동시 구매 전략을 짜는 것이 좋아보인다.